In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#Data Import

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [0]:
datapath = '/content/gdrive/My Drive/Datasets/DogCat/training_data.npy'
training_data = np.load(datapath, allow_pickle=True)

# Model Building

In [0]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, 5)
    self.conv2 = nn.Conv2d(32, 64, 5)
    self.conv3 = nn.Conv2d(64, 128, 5)
    

    x = torch.randn(50,50).view(-1,1,50,50)
    self.to_linear = None
    self.convs(x)

    self.fc1 = nn.Linear(self.to_linear, 512)
    self.fc2 = nn.Linear(512, 1024)
    self.fc3 = nn.Linear(1024, 2)

  def convs(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
    

    #print(x[0].shape)
    if self.to_linear == None:
      self.to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
    return x

  def forward(self, x):
    x = self.convs(x)
    x = x.view(-1, self.to_linear)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.softmax(self.fc3(x), dim=1)
    return x



In [16]:
VAL_PCT = 0.07
test_len = int(len(training_data)*VAL_PCT)

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

train_X = X[:-test_len]
train_y = y[:-test_len]

test_X = X[-test_len:]
test_y = y[-test_len:]

print(len(train_X), len(test_X))

23200 1746


#Training

In [0]:
def train(net, EPOCHS=10):
  loss_function = nn.MSELoss()
  optimizer = optim.Adam(net.parameters(), lr=0.001)
  batch = 100

  for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), batch)):
      batch_X = train_X[i:i+batch].view(-1,1,50,50)
      batch_y = train_y[i:i+batch]

      net.zero_grad()
      outputs = net(batch_X)
      loss = loss_function(outputs, batch_y)
      loss.backward()
      optimizer.step()

  print(loss)

#Testing

In [0]:
def test(net):
  correct = 0
  total = 0

  with torch.no_grad():
    for i in range(len(test_X)):
      realclass = torch.argmax(test_y[i])
      output = net(test_X[i].view(-1,1,50,50))[0]
      predicted = torch.argmax(output)
      if predicted == realclass:
        correct += 1
      total += 1

  print("Accuracy", round(correct/total,4))

In [0]:
net = Net()

In [29]:
train(net, EPOCHS=10)

100%|██████████| 232/232 [01:00<00:00,  3.82it/s]

tensor(0.0699, grad_fn=<MseLossBackward>)


In [30]:
test(net)

Accuracy 0.7692
